In [3]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from collections import defaultdict

In [4]:
class GapTracker:
    def __init__(self):
        self.gap = {}
        self.gaps = defaultdict(list)
        
    def start(self, i, name):
        if name not in self.gap.keys():
            self.gap[name] = {"start": i}
            
    def end(self, i, name):
        if name in self.gap.keys():
            self.gap[name]["end"] = i
            self.gaps[name].append(self.gap[name])
            del self.gap[name]

In [8]:
gt = GapTracker()
gt.start(0, "query")
gt.end(1, "query")
gt.end(2, "sbjct")
gt.start(3, "sbjct")
gt.end(1, "sbjct")
gt.gaps

defaultdict(list,
            {'query': [{'end': 1, 'start': 0}],
             'sbjct': [{'end': 1, 'start': 3}]})

In [9]:
target = "mycobacterium"
strain = "g062726"

In [10]:
# filter trace result
traceFilepath = "./out/{}/{}.trace".format(target, strain)
trace_df = pd.read_csv(traceFilepath)
filtered_df = trace_df[(trace_df["qlength"] >= 300) & (trace_df["possess"] == 0) &(trace_df["max_length"] >= trace_df["qlength"]*0.5)]
print(filtered_df.shape)
filtered_df.head()

(24, 9)


,family,query,qlength,coverage,distance,hit_count,max_length,longest,possess
194,family2371,g063019-cds4882,831,1.000000,0.41589,1,831.0,g062726-NC_002944.2;3968007;3968839;1,0
195,family2380,g063019-cds2115,717,0.998605,0.41589,5,716.0,g062726-NC_002944.2;2245733;2246448;0,0
198,family2419,g063019-cds213,996,0.916667,0.41589,3,913.0,g062726-NC_002944.2;214877;215793;1,0
199,family2434,g063019-cds4180,1395,0.840860,0.41589,4,786.0,g062726-NC_002944.2;4717150;4717941;0,0
202,family2468,g063019-cds2960,681,0.725404,0.41589,6,379.0,g062726-NC_002944.2;1392093;1392472;0,0


In [11]:
query_lst = list(filtered_df["query"])

In [ ]:
def parse_align(fp):
    rec_lst = [rec for rec in SeqIO.parse(fp, "fasta")]
    assert len(rec_lst) == 2
    query = rec_lst[0].seq
    sbjct = rec_lst[1].seq
    assert len(query) == len(sbjct)
    return query, sbjct

In [21]:
for query in query_lst:
    print("START: {}".format(query))
    
    # load query and sbjct
    alignFilepath = "./align/{}/{}/{}.align".format(target, strain, query)
    query, sbjct = parse_align(alignFilepath)
    length = len(query)

    if is_typical(Seq(str(query).replace('-', ''))):

        qi = 0 #record query index
        codon  = {"query": "", "sbjct": ""}
        gt = GapTracker()
        stopCount = 0
        isValid = True
        
        for i in range(length):
            if query[i] != '-' and sbjct[i] != '-': # if alignment found 
                qi += 1
                codon["query"] += query[i]
                codon["sbjct"] += sbjct[i]
                gt.end(i, "query")
                gt.end(i, "sbjct")
            else:
                isValid = False 
                if query[i] == '-':
                    gt.start(i, "query")
                    gt.end(i, "sbjct")
                elif sbjct[i] == '-':
                    gt.start(i, "sbjct")
                    gt.end(i, "query")
                    
            if qi > 0 and qi%3 == 0:
                if isValid:
                    assert len(codon["query"]) == 3
                    assert len(codon["sbjct"]) == 3
                    aa = {"query":  Seq(codon["query"]).translate(),
                             "sbjct": Seq(codon["sbjct"]).translate()}
                    if aa["query"]!='*' and aa["sbjct"]=='*':
                        stopCount += 1
                isValid = True
                codon  = {"query": "", "sbjct": ""}
                
        gt.end(length, "query")
        gt.end(length, "sbjct")
        
        # report stats
        print("\tFOUND: {} additional stop codons".format(stopCount))
        
        for gap in gt.gaps["query"]:
            if gap["start"]!=0 and  gap["end"]!=length:
                print("\tFOUND: {} bp insertion at {}".format(gap["end"]-gap["start"], gap["start"]))
                
        for gap in gt.gaps["sbjct"]:
            if gap["start"]!=0 and  gap["end"]!=length:
                print("\tFOUND: {} bp deletion at {}".format(gap["end"]-gap["start"], gap["start"]))

        print()

START: g063019-cds4882
	FOUND: 0 additional stop codons
	FOUND: 1 bp insertion at 326

START: g063019-cds2115
	FOUND: 1 additional stop codons
	FOUND: 1 bp deletion at 476

START: g063019-cds213
	FOUND: 0 additional stop codons
	FOUND: 3 bp insertion at 215
	FOUND: 3 bp insertion at 666
	FOUND: 3 bp deletion at 852

START: g063019-cds4180
	FOUND: 1 additional stop codons
	FOUND: 2 bp insertion at 170
	FOUND: 6 bp insertion at 502
	FOUND: 3 bp deletion at 759

START: g063019-cds2960
	FOUND: 0 additional stop codons

START: g063019-cds4491
	FOUND: 0 additional stop codons
	FOUND: 2 bp insertion at 315

START: g063019-cds275
	FOUND: 1 additional stop codons

START: g089074-cds5601
	FOUND: 0 additional stop codons
	FOUND: 3 bp insertion at 13
	FOUND: 1 bp deletion at 242

START: g063019-cds4655
	FOUND: 0 additional stop codons
	FOUND: 1 bp deletion at 160

START: g063019-cds3320
	FOUND: 0 additional stop codons
	FOUND: 3 bp insertion at 250
	FOUND: 2 bp deletion at 232

START: g063019-cds2

In [15]:
queryCodon

'atgact'

In [16]:
sbjctCodon

'atgcct'

In [34]:
Seq(str(query).replace('-', '')).translate()

Seq('MTEALRELSGAWNFRDVSDGAPALKPGRLFRSGELSGLDDDGRATLSRLGITDV...LA*', HasStopCodon(ExtendedIUPACProtein(), '*'))

In [33]:
query.translate()

/home/mitsuki/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/Bio/Seq.py:2095: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


TranslationError: Codon 'GA-' is invalid

In [13]:
def is_typical(seq):
    if len(seq)%3 != 0:
        print("WARN: the length is not multiple of 3.")
        return False
    if seq[-3:].translate()!="*":
        print("WARN: the last codon is not stop codon.")
        return False
    if '*' in seq[:-3].translate():
        print("WARN: in-frame stop codon.")
        return False
    return True

In [43]:
Seq(s)[:-3]

Seq('ATGTGA', Alphabet())

In [45]:
s = "ATGTGATGA"
is_typical(Seq(s))

WARN: in-frame stop codon.


False

In [48]:
stopCount = 0
for codon in codon_lst:
    if Seq(codon).translate()=="*":
        stopCount += 1
print("FOUND: {} stop codons".format(stopCount))

FOUND: 1 stop codons


In [31]:
for gap in gt.gaps[0]:
    if gap["start"]==0 or gap["end"]==length:
        pass
    else:
        print("FOUND: {} bp insertion".format(gap["end"]-gap["start"]))
        
for gap in gt.gaps[1]:
    if gap["start"]==0 or gap["end"]==length:
        pass
    else:
        print("FOUND: {} bp deletion".format(gap["end"]-gap["start"]))

FOUND: 1 bp insertion


In [28]:
gap_lst = []
for gaps in gt.gaps:
    for gap in gaps:
        if gap["start"]==0 or gap["end"]==length:
            pass
        else:
            gap_lst.append(gap)

In [29]:
gap_lst

[{'end': 327, 'start': 326}]

In [1]:
length

NameError: name 'length' is not defined

In [70]:
codon_lst[0]

'gcg'

MPEALRELSGAWNFRDVADGAPMLRPGRLFRSGELSGLDDEGRATLRRLGITDVADLRAAREVARRGPGRVPDGVEVHLLPFPDLGEHEAGTDDQAPHEHAFQRLLTGGAEQSAESVDEAATRYMIDEYRQFPTRNGAQRALHRVISLLGAGRAVLTHCFAGKDRTGFVVATVLEAIGVDRDVIVADFLRSNDAAPALRAQISAMIAQRQDTELTPEVVTWTEARLSDGVLGVREEYLAAARQTIDEKFGSLQAYLRDAGVGEADVQRLRAALLA*

In [77]:
def get_six_frame(dnaSeq):
    """
    get 6 frame translation of qseq_dna
    """
    qtrans_lst=[]
    for strand in [1,-1]:
        for gap in range(3):
            start, end = gap, len(dnaSeq)
            while end%3 != gap:
                end -= 1
            if strand == 1:
                subSeq = dnaSeq[gap:end]
            elif strand == -1 :
                subSeq = dnaSeq[gap:end].reverse_complement()
            qtrans_lst.append(subSeq.translate(table=11))
    return qtrans_lst

In [76]:
fnaFilepath = "/home/mitsuki/altorf/denovo/trg-trace/align/mycobacterium/g062726/g063019-cds4882.fna"
rec_lst = [rec for rec in SeqIO.parse(fnaFilepath, "fasta")]

In [78]:
for seq in get_six_frame(rec_lst[0].seq):
    print(seq)

MTEALRELSGAWNFRDVSDGAPALKPGRLFRSGELSGLDDDGRATLSRLGITDVADLRAAREVARRGPGLVPDGVEVHLLPFPDLGEQDAGTDDAAPHEHAFQRLLTGEGVEESEQSVNEAATRYMIDEYRQFPTRNGAQRALHRVISLLADGHSVLTHCFAGKDRTGFVVATVLEAIGIDRDTILADFLRSNDAAPALRAQISAMIAQRQDAELTPEVVTWTEARLSDGVLGVREVYLAAARQTIDEEFGSLDAYLRAASVSESDVERLREALLA*
*LRRCENCRARGTFVTSPTAHRRSNPGGCSAPAN*AGSTTTAARR*AGWASPTSPICGRPARSRGAGRGWFPTASRCTCCPSPISASRTPGPTTRRRTNTPSSACSPARASRNRSSRSTRPPPAT*STNTANSQRVTGRSGRCTASSRCWPTGIRC*RTASPARTAPASWWRPCWRRSASTATPSWPTSCAATTPRPPCGRRSPR*SRSARTPS*PRRW*PGPRRGCPTACSGSARSTWPPRAKPSTRNSGRWTPTCAPRASASQTWSACARRCSP
D*GVARTVGRVELS*RLRRRTGAQTRAAVPLRRTERARRRRPRDAEPAGHHRRRRSAGGPRGRAARAGAGSRRRRGAPVALPRSRRAGRRDRRRGAARTRLPAPAHRRGRRGIGAVGQRGRHPLHDRRIPPIPNA*RGAAGVAPRHLAAGRRAFGADALLRRQGPHRLRGGDRAGGDRHRPRHHPGRLPAQQRRRARPAGADLRDDRAAPGRRADPGGGDLDRGAAVRRRARGPRGLPGRRAPNHRRGIRVAGRLLARRERQRVRRGAPARGAARL
SGEQRLAQALHV*LADARGAQVGVQRPEFLVDGLARGGQVDLADPEHAVGQPRLGPGHHLRGQLGVLALRDHRGDLRPQGGRGVVAAQEVGQDGVAVDADRLQHGRHHEAGAVLAGEAVRQHRMPVGQQRDDAVQRPLRPVTRWELAVFVDHVAGGGLVDRLLRFLDA

In [79]:
for seq in get_six_frame(rec_lst[1].seq):
    print(seq)

MPEALRELSGAWNFRDVADGAPMLRPGRLFRSGELSGLDDEGRATLRRLGITDVADLRAAREVARRGPGRVPDGVEVHLLPFPDLGEHEAGTDDQAPHEHAFQRLLTGDGGRAVGGVRRRGRHPLHDRRIPAIPNA*RGAASAAPGHLAAGRRACGAHPLLRRQGPHRVRGGDGARSDRRRPRRHRRRLSAQQRRRARAARADLGDDRAAPGHRADPGGGDLDRGAAVRRCAGGARGVPGRRSANHRREVRVAAGLPARRRGRRGRRATPARGAARL
CLRRCENCPARGTFVTSPTVRPCCGRAGCSGPAS*AGSTTRAARRCAGWASPTSPTCARPARWPGAARAGFPTGSRCTCCPFPISASTRPAPTTRRRTSTPSSGCSPATGAEQSAESVDEAATRYMIDEYRQFPTRNGAQRALHRVISLLGAGRAVLTHCFAGKDRTGFVVATVLEAIGVDRDVIVADFLRSNDAAPALRAQISAMIAQRQDTELTPEVVTWTEARLSDGVLGVREEYLAAARQTIDEKFGSLQAYLRDAGVGEADVQRLRAALLA*
A*GAARTVRRVELS*RRRRCAHAAAGPAVPVRRAERARRRGPRDAAPAGHHRRRRPARGPRGGPARPGPGSRRGRGAPAALSRSRRARGRHRRPGAARARLPAAAHRRRGPSSRRSPSTRPPPAT*STNTGNSQRVTGRSERCTGSSRCWAPGVRCSPTASPARTAPGSWWRRCSKRSASTATSSSPTFCAATTPRPRCARRSRR*SRSARTPS*PRRW*PGPRRGCPTVCWGCARSTWPPLGKPSTRSSGRCRPTCATPGSARQTCNACARRCSP
QASSAARRRCTSASPTPASRR*ACSDPNFSSMVCRAAARYSSRTPSTPSDSRASVQVTTSGVSSVSWRCAIIAEICARSAGAASLLRRKSATMTSRSTPIASSTVATTNPVRSLPAKQWVSTARPAPSSEMTRCSARCAPLRVGNCRYSSIM*RVAASSTDSADCSA

In [9]:
def get_rec(target, family, orfid, ext):
    assert ext=="fna" or ext=="faa"
    
    fp = "/data/mitsuki/data/denovo/{0}/annotation/refseq/family/{2}/{1}.{2}".format(target, family, ext)
    for rec in SeqIO.parse(fp, "fasta"):
        if rec.id == orfid:
            return rec
    return None

In [ ]:
family6623
query = "g063032-cds2443"

In [11]:
get_rec("mycobacterium", "family6623", "g063032-cds2443", "faa") #当然1フレーム目の翻訳産物

SeqRecord(seq=Seq('MAGGTTKANCSQKCEAGGDRRIRIAARQHGSRLPSVHFGAAWVGLGAFATLPAA...GLP', SingleLetterAlphabet()), id='g063032-cds2443', name='g063032-cds2443', description='g063032-cds2443 WP_080674231.1 hypothetical protein [Mycobacterium kansasii]', dbxrefs=[])

In [3]:
seqFilepath = "../helper/extract.out"
rec_lst = []
for rec in SeqIO.parse(seqFilepath, "fasta"):
    rec_lst.append(rec)

In [4]:
rec_lst

[SeqRecord(seq=Seq('AGCGGGCGACGTTCCGAATGTTCGGTCCGCCGCCCGCTAGCACGGGACTCGGTT...GTG', SingleLetterAlphabet()), id='g062726-NC_002944.2:[438217,', name='g062726-NC_002944.2:[438217,', description='g062726-NC_002944.2:[438217, 438511),0', dbxrefs=[]),
 SeqRecord(seq=Seq('ATAGCTGGCGGAACTACAAAAGCAAATTGTTCGCAGAAATGCGAAGCGGGCGGC...TGA', SingleLetterAlphabet()), id='g063032-cds2443', name='g063032-cds2443', description='g063032-cds2443 lcl|NC_022663.1_cds_WP_080674231.1_2445 [locus_tag=MKAN_RS29640] [db_xref=GeneID:31546045] [protein=hypothetical protein] [protein_id=WP_080674231.1] [location=complement(2811380..2811718)] [gbkey=CDS]', dbxrefs=[])]

In [ ]:
AGDVPNVRSAAR*HGTRLPSVHLAAWVGLGDLTTLPVAAPKGP*TAIRNYAGPQHFCHFRGYDSAWVPSTVLGARIGRSNLLFRDLGLTGLP

In [5]:
for seq in get_six_frame(rec_lst[0].seq):
    print(seq)

SGRRSECSVRRPLARDSVTKRASRGCVGWPRRSHDASGCSPTQRAVDTFRNSQLRRPATLLPFSRL*LRVGAEHRAGRPDREIEPSLPDRPWPHRASV
AGDVPNVRSAAR*HGTRLPSVHLGAAWVGLGDLTTLPVAAPPKGP*TPSAIRNYAGPQHFCHFRGYDSAWVPSTVLGARIGRSNLLFRIDLGLTGLP
RATFRMFGPPPASTGLGYQACISGLRGLASAISRRFRLQPHPKGRRHLPQFAITQARNTSAIFAAMTPRGCRAPCWAPGSGDRTFSSGSTLASPGFR
HGSPVRPRSIRKRRFDLPIRAPSTVLGTHAES*PRKWQKCCGPA*LRIAEGVYGPLGGAATGSVVRSPRPTHAAPRCTLGNRVPC*RAADRTFGTSPA
RKPGEAKVDPEEKVRSPDPGAQHGARHPRGVIAAKMAEVLRACVIANCGRCLRPFGWGCNRKRREIAEANPRSPEMHAW*PSPVLAGGGPNIRNVAR
TEAR*GQGRSGREGSISRSGRPARCSAPTRSHSRENGRSVAGLRNCELRKVSTALWVGLQPEAS*DRRGQPTQPRDARLVTESRASGRRTEHSERRP


In [6]:
for seq in get_six_frame(rec_lst[1].seq):
    print(seq)

IAGGTTKANCSQKCEAGGDRRIRIAARQHGSRLPSVHFGAAWVGLGAFATLPAAAPPKGPSLPSALRNYAGPQHFCHYRGYDSAWVPGTVLGARVGRTNLLFRIDLGLTGLP*
*LAELQKQIVRRNAKRAATEGFESPPANTVLGYQACISGLRGLASALLRHFRLQPHPKDRRYHLHFAITQARNTSAIIAAMTPRGCREPCWAPGSGERTFSSGSTLASPGFR
SWRNYKSKLFAEMRSGRRPKDSNRRPPTRFSVTKRAFRGCVGWPRRFCDTSGCSPTQRTVATICTSQLRRPATLLPLSRL*LRVGAGNRAGRPGRENEPSLPDRPWPHRASV
SRKPGEAKVDPEEKVRSPDPGAQHGSRHPRGVIAAIMAEVLRACVIAKCRW*RRSFGWGCSRKCRKSAEANPRSPEMHAW*PRTVLAGGDSNPSVAARFAFLRTICFCSSASY
TEAR*GQGRSGREGSFSRPGRPARFPAPTRSHSRDNGRSVAGLRNCEVQMVATVLWVGLQPEVSQKRRGQPTQPRNARLVTENRVGGRRFESFGRRPLRISANNLLL*FRQL
HGSPVRPRSIRKRRFVLPTRAPSTVPGTHAES*PR*WQKCCGPA*LRSADGSDGPLGGAAAGSVAKAPRPTHAAPKCTLGNREPCWRAAIRILRSPPASHFCEQFAFVVPPA


In [9]:
print(get_six_frame(rec_lst[1].seq)[0])

IAGGTTKANCSQKCEAGGDRRIRIAARQHGSRLPSVHFGAAWVGLGAFATLPAAAPPKGPSLPSALRNYAGPQHFCHYRGYDSAWVPGTVLGARVGRTNLLFRIDLGLTGLP*
